## Setup

In [107]:
# ignore NumbaDeprecationWarning
import numba
import warnings
warnings.filterwarnings("ignore", category=numba.NumbaDeprecationWarning)

from bertopic import BERTopic
#from sklearn.datasets import fetch_20newsgroups

import csv
import re

from nltk.corpus import stopwords

from hdbscan import HDBSCAN

import pandas as pd

## Import and Clean Data

In [108]:
# Import Businesses TSV as list of strings
with open('all_businesses.tsv', newline='') as f:
    reader = csv.reader(f, delimiter='\t')
    docs = [item.replace('\xa0', ' ') for sublist in reader for item in sublist]

# Remove punctuation
docs = [re.sub(r'[^\w\s]', '', doc) for doc in docs]

# Lowercase
docs = [doc.lower() for doc in docs]

# Remove stopwords
german_stop_words = stopwords.words('german')

# Import custom stopwords file as list of strings
with open('../../data/custom_stopwords.txt', 'r') as f:
   custom_stopwords = f.readlines()

# remove whitespace characters like `\n` at the end of each line
custom_stopwords = [x.strip() for x in custom_stopwords]

# remove stopwords from docs
docs = [' '.join(word for word in doc.lower().split() if word not in german_stop_words) for doc in docs]
docs = [' '.join(word for word in doc.lower().split() if word not in custom_stopwords) for doc in docs]

# remove "na" from docs
docs = [doc for doc in docs if doc != "na"]

In [109]:
# Insepct Data

# print head of docs
print(docs[:2])

# print size of docs
print(len(docs)) # 18846

['schkg schutz gutgläubiger erwerber 23 2000 reichte nationalrat jeanmichel cina parlamentarische gutgläubige erwerber schützen konkursiten konkurseröffnung deren publikation beziehungsweise anmerkung grundbuch grundstück erwirbt 204 1 298 2 bundesgesetzes schuldbetreibung konkurs schkg seien entsprechend revidieren rechtsfragen nationalrates beantragte 23 2001 9 8 stimmen geben entgegen antrag gab nationalrat 15 2001 anschliessend arbeitete vorliegenden entwurf gesetzesänderung wobei inhaltlich geändert wurde anstatt vorsah immobiliarsachenrechtlichen gutglaubensschutz konkursbeschlag vorgehen lassen frist konkurseröffnung deren anmerkung grundbuch kurz möglich gehalten vorrang konkursbeschlages dagegen geändert beantragte folgende 176 2 schkg konkurs spätestens tage eröffnung grundbuch anzumerken stimmte beantragte jedoch regelung nachlassstundung beschliessen 296 zweiter satz schkg folgt ändern nachlassstundung spätestens tage bewilligung grundbuch anzumerken gutgläubiger erwerber s

## Modelling

In [130]:
# https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#hdbscan
hdbscan_model = HDBSCAN(
    min_cluster_size = 2,
    metric = 'euclidean',
    prediction_data = True)


# BERTopic German model
# Parameter tuning: https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#bertopic
topic_model = BERTopic(
    language = "multilingual",
    min_topic_size = 2,
    verbose = True,
    top_n_words = 20,
    n_gram_range = (1, 2),
    #calculate_probabilities = True, # turn on later again to calc probs
    hdbscan_model=hdbscan_model,
    embedding_model = "distiluse-base-multilingual-cased-v1") # https://www.sbert.net/docs/pretrained_models.html
#.fit(docs)

topics, probs = topic_model.fit_transform(docs)

Batches: 100%|██████████| 21/21 [01:50<00:00,  5.28s/it]
2023-07-25 18:09:08,298 - BERTopic - Transformed documents to Embeddings
2023-07-25 18:09:11,270 - BERTopic - Reduced dimensionality
2023-07-25 18:09:11,320 - BERTopic - Clustered reduced embeddings


In [131]:
# Number of topics
topic_info = topic_model.get_topic_info()
num_topics = topic_info.shape[0]
print(f"There are {num_topics} topics.")

There are 86 topics.


## Print Results

In [85]:
print("topic_model.get_topic_info()")
print(topic_model.get_topic_info())

topic_model.get_topic_info()
    Topic  Count                                               Name  \
0      -1    111           -1_schweiz_botschaft_recht_übereinkommen   
1       0     38  0_informationsaustausch_doppelbesteuerungsabko...   
2       1     18                   1_meldestelle_schweiz_banken_snb   
3       2     15  2_verkehr_rpv_verkehrssicherheit_schwerverkehr...   
4       3     14  3_elektronische_elektronischen_signatur_person...   
..    ...    ...                                                ...   
83     82      2  82_quecksilber_feinstaub_schwermetalle_verringern   
84     83      2  83_terrorismus_bekämpfung terrorismus_bekämpfu...   
85     84      2  84_handelsregister_bodeninformationssystem_gru...   
86     85      2       85_aufsichtsabgabe_64c_alv_aufsichtsbehörden   
87     86      2  86_uk_sma_dienstleistungserbringern_dienstleis...   

                                       Representation  \
0   [schweiz, botschaft, recht, übereinkommen, änd...   
1   

In [51]:
print("topic_model.get_topic(0)")
print(topic_model.get_topic(0))

topic_model.get_topic(0)
[('informationsaustausch', 0.06903427671805619), ('doppelbesteuerungsabkommen', 0.059208786419129536), ('dba', 0.05799066336039875), ('doppelbesteuerung', 0.05738622339487173), ('abkommen', 0.051573464671800406), ('dividenden', 0.03303146027793287), ('interessierten', 0.031403557511010524), ('änderungsprotokoll', 0.031314699169353005), ('wirtschaftskreise', 0.030739057081380777), ('abschluss', 0.027916721534702392)]


In [132]:
# extract information on a document level
#print("topic_model.get_document_info(docs)")
#print(topic_model.get_document_info(docs))

# save document level information to csv
doc_level_info = topic_model.get_document_info(docs)

#doc_level_info.to_csv('doc_level_info.csv', index=False)

# save doc_level_info to csv
# with open('doc_level_info.csv', 'w', newline='') as f:
#     writer = csv.writer(f)
#     writer.writerows(doc_level_info)


pandas.core.frame.DataFrame

In [106]:
# Get the unique values in the "Representation" column

unique_values_representation = doc_level_info['Representation']

type(unique_values_representation)



pandas.core.series.Series

## Topic Distribution

In [66]:
#topic_distr, _ = topic_model.approximate_distribution(docs)
#print(topic_distr)

# print dimension of probs
#print(probs.shape)

(672, 59)


In [67]:
# Export topic_distribution as CSV
with open('topic_distribution_test.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    #writer.writerows(topic_distr)
    writer.writerows(probs)

## Hierarchical Topic Modeling

In [84]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

100%|██████████| 86/86 [00:00<00:00, 117.25it/s]

.
├─wähler sollten_bewusst stimme_stimme unterschied_sollten bewusst_wähler
│    ├─bewusst stimme_wähler sollten_sollten bewusst_stimme unterschied_wähler
│    │    ├─finanzwesen_finanzwesen finanzwesen_sollten bewusst_bewusst stimme_wähler sollten
│    │    │    ├─■──doppelbesteuerung_finanzwesen_steuerrecht internationale_finanzwesen steuerrecht_steuerabkommen ── Topic: 4
│    │    │    └─■──finanzwesen_finanzwesen finanzwesen_stimme unterschied_sollten bewusst_bewusst stimme ── Topic: 7
│    │    └─stimme unterschied_wähler sollten_sollten bewusst_bewusst stimme_wähler
│    │         ├─bewusst stimme_stimme unterschied_sollten bewusst_wähler sollten_wähler
│    │         │    ├─stimme unterschied_sollten bewusst_wähler sollten_bewusst stimme_wähler
│    │         │    │    ├─■──wähler sollten_sollten bewusst_stimme unterschied_bewusst stimme_gesundheit ── Topic: 37
│    │         │    │    └─■──wähler sollten_sollten bewusst_stimme unterschied_bewusst stimme_wähler ── Topic: 8
│    